# Chapter 3

## Figure 3.03 Michaelis-Menten kinetics

In [ ]:
using DifferentialEquations
using ModelingToolkit
using Plots
Plots.gr(lw=2)

In [ ]:
hill(x, k) = x / (x + k)
hill(x, k, n) = hill(x^n, k^n)

In [ ]:
@parameters eT k1 km1 k2
@variables t S(t) ES(t) P(t) eFree(t) v1(t) v2(t)
D = Differential(t)

In [ ]:
@named fullSys = ODESystem([
    eFree ~ eT - ES,
    v1 ~ k1 * S * eFree - km1 * ES,
    v2 ~ k2 * ES,
    D(S) ~ -v1,
    D(ES) ~ v1 - v2,
    D(P) ~ v2
])

In [ ]:
fullSys = structural_simplify(fullSys)

In [ ]:
# Apply QSSA on the ES complex to somplify the model
@named simpSys = ODESystem([
    D(S) ~ -k2 * eT * hill(S, (km1 + k2) / k1),
])

In [ ]:
u0 = [S => 5.0, ES => 0.0, P => 0.0]
tend = 1.0
params = [eT => 1.0, k1 => 30.0, km1 => 1.0, k2 => 10.0]
sol = solve(ODEProblem(fullSys, u0, tend, params))

In [ ]:
p1 = plot(sol, vars=[S, ES, P, eFree], xlabel="Time (AU)", ylabel="Concentration (AU)", legend=:right)

In [ ]:
solsimp = solve(ODEProblem(simpSys, u0, tend, params))

In [ ]:
p2 = plot(sol, vars=[1, 3], line=(:dash), label=["S (full)" "P (full)"])
plot!(p2, solsimp, line=(:blue), lab="S (reduced)")
plot!(p2, solsimp, vars=((t, s)->(t, 5.0 - s), 0, 1), line=(:red), lab="P (reduced)")
plot!(p2, xlabel="Time (AU)",  ylabel="Concentration (AU)", xlims=(0.0,1.0), ylims=(0.0,5.0), legend = :right)

## Problem 3.7.5 Reduced Michaelis-Menten

In [ ]:
using Plots
using DifferentialEquations
using ModelingToolkit
Plots.gr(lw=2, fmt=:png)

In [ ]:
# Convenience functions
hill(x, k) = x / (x + k)
hill(x, k, n) = hill(x^n, k^n)
reducedmm(x, k) = x / k

In [ ]:
@parameters v_0 v_m1 v_m2 v_m3 Km_1 Km_2 Km_3
@variables t S1(t) S2(t) S3(t) v1(t) v2(t) v3(t)
D = Differential(t)

In [ ]:
eqsFull = [ v1 ~ v_m1 * hill(S1, Km_1),
            v2 ~ v_m2 * hill(S2, Km_2),
            v3 ~ v_m3 * hill(S3, Km_3),
            D(S1) ~ v_0 - v1,
            D(S2) ~ v1 - v2,
            D(S3) ~ v2 - v3]

@named fullsys = ODESystem(eqsFull)
fullSys = structural_simplify(fullsys)

In [ ]:
eqsRe =   [ v1 ~ v_m1 * S1 / Km_1,
            v2 ~ v_m2 * S2 / Km_2,
            v3 ~ v_m3 * S3 / Km_3,
            D(S1) ~ v_0 - v1,
            D(S2) ~ v1 - v2,
            D(S3) ~ v2 - v3]
@named reSys = ODESystem(eqsRe)
reSys = structural_simplify(reSys)

In [ ]:
u0 = [S1 => 0.3, S2 => 0.2, S3 => 0.1]
tend = 2.0
params = [v_0 => 2, v_m1 => 9, v_m2 => 12, v_m3 => 15, Km_1 => 1.0, Km_2 => 0.4, Km_3 => 3.0]

In [ ]:
sol1 = solve(ODEProblem(fullSys, u0, tend, params))

plot(sol1, ylims=(0.0, 0.8),
     title="Problem 3.7.5 (1)",
     xlabel="Time (arbitrary units)",
     ylabel="Concentration (arbitrary units)")

In [ ]:
u1 = [S1=>6.0, S2=>4.0, S3=>4.0]
sol2 = solve(ODEProblem(fullSys, u1, 4.0, params))

plot(sol2, ylims=(0.0, 6.0),
     title="Problem 3.7.5 (2)",
     xlabel="Time (arbitrary units)",
     ylabel="Concentration (arbitrary units)")

In [ ]:
sol3 = solve(ODEProblem(reSys, u0, tend, params))

p3 = plot(sol1, ylims=(0.0, 0.8),
     title="Problem 3.7.5 (1) (full vs reduced)",
     xlabel="Time (arbitrary units)",
     ylabel="Concentration (arbitrary units)",
     labels=["S1 " "S2 " "S3 "], ls=:dash)
plot!(p3, sol3, labels=["S1 (reduced)" "S2 (reduced)" "S3 (reduced)"] )

In [ ]:
sol4 = solve(ODEProblem(reSys, u1, 4.0, params))
p4 = plot(sol2, ylims=(0.0, 8.0),
     title="Problem 3.7.5 (2) (full vs reduced)",
     xlabel="Time (arbitrary units)",
     ylabel="Concentration (arbitrary units)",
     labels=["S1 " "S2 " "S3 "], ls=:dash)
plot!(p4, sol4, labels=["S1 (reduced)" "S2 (reduced)" "S3 (reduced)"] )